## Constrained docking protocol

In this tutorial, we will demonstrate how you can use `rush-py` to conduct a large-scale virtual screen on a target using a constrained docking protocol.

We will use the Zinc20 database of FDA approved drugs as our sample ligand database, but Rush's capability means that this protocol could scale to screen tens of millions of ligands.

## 0.0) Imports

In [1]:
import requests
import csv
import shutil
import json
import numpy as np
from pathlib import Path
from enum import Enum

from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import AllChem, SDMolSupplier
from rdkit.Chem import rdFMCS, rdRascalMCES
from rdkit.Chem import rdDistGeom
from rdkit import Chem
from rdkit.Chem.rdMolAlign import AlignMol
from rdkit.Chem import rdForceFieldHelpers

from typing import List, Optional

## 1.1) Configuration

In [2]:

# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/constrained-docking/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)

In [3]:
# Define our project information
DESCRIPTION = "rush-py constrained docking protocol"
TAGS = ["qdx", "rush-py-v2", "demo", "contrained-docking"]
WORK_DIR = Path.home() / "qdx" / "constrained-docking"

In [5]:
import rush
import asyncio

client = await rush.build_provider_with_functions(
   url="https://tengu-server-staging-edh4uref5a-as.a.run.app/",
)

## 0.1) Virtual screen configuration
The expectation for using this virtual screen is that you will have a protein with a characterized binding ligand and you will want to try to dock a set of alternate targets to the protein.
You should save files to the same directory that you run this notebook in.

The key configuration items are:

`REFERENCE_LIGAND_FILEPATH`: the path to a SDF file containing a known binder for your protein target.

`PROTEIN_TARGET_FILEPATH`: the path to a PDB file containing your protein target. Please note that the binder must be removed. 

`VIRTUAL_SCREEN_LIBRARY_URL`: a URL to a Zinc20 csv virtual screen database download.

`SORT_BY`: sort the gnina minimised poses output by a particular score. For more information on GNINA outputs, see [here](https://github.com/gnina/gnina).


## 0.2) 
For this example, we will fetch an example protein and ligand from RCSB. We are using CDK with the JWS648 inhibitor as a known ligand to serve as our template.

In [3]:
class SortKey(Enum):
    CNN_SCORE = "cnn_score"
    AFFINITY =  "affinity"
    CNN_AFFINITY = "cnn_affinity"

REFERENCE_LIGAND_FILEPATH = Path.cwd() / "3pxy_B_JWS.sdf" 
PROTEIN_TARGET_FILEPATH = Path.cwd() / "3pxy_cleaned.pdb"
VIRTUAL_SCREEN_LIBRARY_URL = 'https://zinc20.docking.org/substances/subsets/fda.csv?count=all'
SORT_BY = SortKey.CNN_SCORE

OBJECT_POSE_FILEPATH = Path.cwd() / 'objects' 

In [13]:
!pdb_fetch '3pxy' |  pdb_delhetatm > 3pxy_cleaned.pdb
REFERENCE_LIGAND_FILEPATH.write_bytes(
    requests.get('https://models.rcsb.org/v1/3pxy/ligand?auth_seq_id=299&label_asym_id=B&encoding=sdf&filename=3pxy_B_JWS.sdf').content
)
!ls

3pxy_B_JWS.sdf	3pxy_cleaned.pdb  data.pkl


## 0.1) Constrained docking code
The below block of code is the set of helpers functions necessary for performing constrained docking as part of a large-scale virtual screen.

In [5]:
def get_mcs(query_ligand: Chem.Mol, reference: Chem.Mol, timeout=20, **kwargs) -> Optional[Chem.Mol]:
    if kwargs.get("ignore_heavy_atom"):
        atom_comparision_method = rdFMCS.AtomCompare.CompareAnyHeavyAtom
    else:
        atom_comparision_method = rdFMCS.AtomCompare.CompareAnyHeavyAtom

    if kwargs.get("ignore_bond_order"):
        bond_comparision_method = rdFMCS.BondCompare.CompareAny
    else:
        bond_comparision_method = rdFMCS.BondCompare.CompareOrder

    if kwargs.get("use_rascal_mces"):
        mcs = rdRascalMCES.FindMCES(
                [reference, query_ligand],
                atomCompare=atom_comparision_method,
                bondCompare=bond_comparision_method
            )

    else:
        mcs = rdFMCS.FindMCS(
            [reference, query_ligand],
            threshold=0.9,
            completeRingsOnly=kwargs.get("complete_rings_only", False),
            atomCompare=atom_comparision_method,
            bondCompare=bond_comparision_method,
            timeout=timeout
            )
    if mcs_result_exists(mcs) and meets_similarity_threshold(mcs, reference):
        return Chem.MolFromSmarts(mcs.smartsString, mergeHs=True)
    return None


def meets_similarity_threshold(mcs: Chem.Mol, reference: Chem.Mol, min_threshold=0.2) -> bool:
    if mcs_result_exists(mcs):
        mcs_mol = Chem.MolFromSmarts(mcs.smartsString, mergeHs=True)
        match_ratio = mcs_mol.GetNumAtoms() / reference.GetNumAtoms()

        return match_ratio >= min_threshold
    
    return False


def get_filtered_mcs(mcss, reference) -> List[Chem.Mol]:
    return [mcs for mcs in mcss if meets_similarity_threshold(mcs, reference)]



INITIAL_FILTER_DISTANCE = 1000
def get_diverse_substructure_matches(reference, mcs_mol, minimum_difference=5) -> List[Chem.Mol]:
    """
    Prunes a list of MCS substructure hits within a molecule and keeps only those which are at least the minimum difference apart
    This is to capture as many unique substructure hits across the query and the ligand, but discard those that are not meaningfully different (e.g. a rotation of the bond) 
    A default of 5 is an effective balance between retaining novel hits and discarding excessively similar values.
    """
    substructures = reference.GetSubstructMatches(mcs_mol, uniquify=False)

    output_structures = [
        substructures[0]
    ]
    for substructure in substructures[1:]:
        distance = INITIAL_FILTER_DISTANCE
        j = 0

        while (distance >= minimum_difference) and j < len(output_structures):
            ref = np.array(output_structures[j])
            distance = sum(np.array(substructure) != ref)
            j += 1

        if distance >= minimum_difference:
            output_structures.append(substructure)
    
    return output_structures

@time_function
def get_tethered_atoms(substruct_match) -> str:
    """
    Return a formatted string of atom indexes to pass to "TETHERED ATOMS" configuration option for rxdock

    Example input:
    (0, 1, 2)

    Example output:
    1,2,3
    """
    return ','.join(str(index + 1) for index in substruct_match)

def mcs_result_exists(mcs) -> bool:
    return mcs.smartsString and len(mcs.smartsString) > 0

@time_function
def get_force_field(geom_calc, query_mol):
    ff = rdForceFieldHelpers.UFFGetMoleculeForceField(query_mol, confId=0)

    for i in geom_calc.coordMap:
        point = geom_calc.coordMap[i]
        point_idx = ff.AddExtraPoint(point.x, point.y, point.z, fixed=True) - 1
        ff.AddDistanceConstraint(point_idx, i, 0, 0, 100.0)
    ff.Initialize()

    return ff

@time_function
def get_template_aligned_pose(query, reference, query_ligand_map, geom_calc, n_tries=5) -> Chem.Mol:
    """
    Align query molecule to "template" (reference molecule)
    Distance constrints are enforced on the input ligand to mould the geometry of its matching substructure to that in the template
    """
    temp_query_mol = Chem.Mol(query) 

    min_energy = np.inf
    bestmol = None
        
    for _ in range(n_tries):
        # Repeat the alignment step at each starting configuration
        # Places internal geometry (bond angles, torsion, etc) of parts of the ligand 
        # that matches that in the reference 
        # Need to error here if ci > 0
        AllChem.EmbedMolecule(temp_query_mol, geom_calc)
        # This step is critical as it places the molecule in a good starting position relative to the reference
        AlignMol(temp_query_mol, reference, atomMap=query_ligand_map)
        
        ff = get_force_field(geom_calc, temp_query_mol)
        
        minimize_tries = 4
        more_to_minimize = ff.Minimize(energyTol=1e-4, forceTol=1e-3)
        while more_to_minimize and minimize_tries:
            more_to_minimize = ff.Minimize(energyTol=1e-4, forceTol=1e-3)
            minimize_tries -= 1
        current_energy = ff.CalcEnergy()
        
        if current_energy < min_energy:
            bestmol = Chem.Mol(
                temp_query_mol
            )
            min_energy = current_energy
        temp_query_mol = Chem.Mol(
            query
        )

    AlignMol(bestmol, reference, atomMap=query_ligand_map)

    return bestmol

@time_function
def get_substructure_matches(molecule, mcs_mol, uniquify=True) -> Chem.Mol:
    return molecule.GetSubstructMatches(mcs_mol, uniquify=uniquify)        

@time_function
def get_dist_geom_calculator(reference, query_matches, reference_match, constrained_atoms):
    geom_calc = rdDistGeom.ETKDGv3()
    geom_calc.trackFailures = True
    geom_calc.coordMap = {
        query_matches[atom_idx]: reference.GetConformer().GetAtomPosition(reference_match[atom_idx]) for atom_idx in range(len(constrained_atoms))
    }

    return geom_calc

@time_function
def get_initial_poses(query, reference, max_symmetry=5) -> List[Chem.Mol]:
    """
    Use maximum common substructure between query and template ligand to generate initial docking poses
    of query ligand

    """
    mcs_mol = get_filtered_mcs(query, reference)
    if not mcs_mol:
        print(f"No match found between {query.smarts} ")
        return []
    reference_matches = get_diverse_substructure_matches(reference, mcs_mol)
    molecule_matches = get_substructure_matches(query, mcs_mol)

    constrained_atoms = molecule_matches[0]
    constrained_atom_ids = get_tethered_atoms(constrained_atoms)

    molhits = query.GetSubstructMatch(
            mcs_mol
        ) 

    poses = []
    
    n_iterations = min(len(reference_matches), max_symmetry)

    for i in range(n_iterations):
        geom_calc = get_dist_geom_calculator(reference, molhits, reference_matches[i], constrained_atoms)
        
        posed_mol = get_template_aligned_pose(
            query,
            reference,
            [(m_idx, r_idx) for m_idx, r_idx in zip(molhits, reference_matches[i])],
            geom_calc
        )
        posed_mol.SetProp("TETHERED ATOMS", constrained_atom_ids)
        poses.append(posed_mol)

    return poses

## 1.0) Virtual screen library
This section contains the section for downloading a virtual screen library from Zinc20 and converting it into RDKit molecules we can constrain

In [ ]:
from rdkit.Chem import MolToInchi
def load_zinc20_virtual_screen_library(url) -> List[Chem.Mol]:
    ligands = []
    with requests.get(url, stream=True) as response:
        response.raise_for_status()

        lines = (line.decode('utf-8') for line in response.iter_lines())
        reader = csv.DictReader(lines)
        
        for row in reader:
            print(row, end=" ")
            if 'smiles' in row:
                mol = MolFromSmiles(row['smiles'])
                mol = Chem.AddHs(mol)
                # For reproducibility. In principle it won't matter much because we're moving it to fit the template
                Chem.rdDistGeom.EmbedMolecule(mol, 1, randomSeed=0xf00d) 

                ligands.append(mol)

    return ligands
    
query_ligands = load_zinc20_virtual_screen_library(VIRTUAL_SCREEN_LIBRARY_URL)

In [6]:
suppl = SDMolSupplier(REFERENCE_LIGAND_FILEPATH)
reference_ligand = suppl[0]
assert reference_ligand is not None 

[01:32:00] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


## 1.1) Generate initial poses
In this stage, we generate initial constrained poses from our query ligands that we will feed to our docking pipeline.

In [20]:
from mpire import WorkerPool
import multiprocessing

poses = []

def process_query_ligand(query_ligand):
    """ Function to process each query ligand with get_initial_poses"""
    print(MolToSmiles(query_ligand))
    return get_initial_poses(query_ligand, reference_ligand)

num_processes = multiprocessing.cpu_count() - 5


with WorkerPool(n_jobs=num_processes, enable_insights=True) as pool:
    poses.extend(pool.map_unordered(process_query_ligand, query_ligands))


[H]C#C[C@]1(O[H])C([H])([H])C([H])([H])[C@@]2([H])[C@]3([H])C([H])([H])C([H])([H])c4c([H])c(OC([H])([H])[H])c([H])c([H])c4[C@@]3([H])C([H])([H])C([H])([H])[C@@]21C([H])([H])[H][H]OC([H])([H])[C@@]([H])(O[H])C([H])([H])Oc1c([H])c([H])c([H])c([H])c1OC([H])([H])[H][H]OC(=O)C([H])([H])N([H])C(=O)[C@]([H])(S[H])C([H])([H])[H][H]c1nc([H])n(C([H])([H])[C@@]2(c3c([H])c([H])c(Cl)c([H])c3Cl)OC([H])([H])[C@]([H])(C([H])([H])Oc3c([H])c([H])c(N4C([H])([H])C([H])([H])N(C(=O)C([H])([H])[H])C([H])([H])C4([H])[H])c([H])c3[H])O2)c1[H]



get_mcs executed in 0.002942 seconds
get_diverse_substructure_matches executed in 0.000462 seconds
get_substructure_matches executed in 0.000038 seconds
get_tethered_atoms executed in 0.000015 secondsget_mcs executed in 0.006194 seconds

get_dist_geom_calculator executed in 0.000218 secondsget_diverse_substructure_matches executed in 0.000408 secondsget_mcs executed in 0.008329 seconds
get_substructure_matches executed in 0.000061 seconds[H]Oc1c([H])c2c(c([H])c1OC([H])(


get_diverse_substructure_matches executed in 0.000347 secondsget_force_field executed in 0.000226 seconds
get_tethered_atoms executed in 0.000018 seconds
get_tethered_atoms executed in 0.000017 seconds


get_substructure_matches executed in 0.000100 seconds
get_dist_geom_calculator executed in 0.000223 secondsget_dist_geom_calculator executed in 0.000222 secondsget_dist_geom_calculator executed in 0.000219 seconds

get_tethered_atoms executed in 0.000018 seconds

get_dist_geom_calculator executed in 0.000220 seconds
get_force_field executed in 0.000311 seconds
get_force_field executed in 0.000146 seconds

get_diverse_substructure_matches executed in 0.000401 seconds
get_substructure_matches executed in 0.000090 seconds
get_force_field executed in 0.000131 secondsget_tethered_atoms executed in 0.000017 seconds
get_dist_geom_calculator executed in 0.000233 seconds

get_force_field executed in 0.000225 seconds
get_force_field executed in 0.000129 seconds
get_force_field executed in 0.000

20

## 1.2) Write poses to SDF files
In this stage we write the poses calculated by our constrained docking into SDF files.

In [ ]:
INITIAL_POSE_PATH = Path.cwd() / "initial_poses"
if os.path.exists(INITIAL_POSE_PATH):
        shutil.rmtree(INITIAL_POSE_PATH)
os.makedirs(INITIAL_POSE_PATH)
os.chdir(INITIAL_POSE_PATH)

In [ ]:
molecule_hashes = {}

for query_poses in poses:
    for query_pose in query_poses:
        mol_hash = hash(query_pose)
        molecule_hashes[mol_hash] = query_pose

        filename = f'{mol_hash}.sdf'
        writer = Chem.SDWriter(filename)
        writer.write(query_pose)
        writer.close()

## 1.3) Run constrained docking via the Rush platform
In this step, we use rxdock and gnina to run our constrained docking workflow.

In [ ]:
rxdock_outputs = []

TETHERED_DOCKING_CONFIGURATION = {
        "rot_mode": "TETHERED",
        "trans_mode": "TETHERED",
        "max_rot": 3.0,
        "max_trans": 1.0
}  
for pose in molecule_hashes:
    (conformers, scores, sdf) = await client.rxdock(
        None,
        None,
        {"n_runs": 10},
        TETHERED_DOCKING_CONFIGURATION 
        None,
        PROTEIN_TARGET_FILEPATH,
        Path.cwd() / f'{pose}.sdf'
    )
    rxdock_outputs.append((sdf, pose))

await asyncio.gather(*(sdf.download(filename=f"{pose}_rxdock.sdf",) for sdf, pose in rxdock_outputs))

2024-05-08 03:06:41,014 - rush - INFO - Argument 1cfde7ae-7035-4ce2-adf3-22e7b318322e is now ModuleInstanceStatus.ADMITTED
2024-05-08 03:06:41,030 - rush - INFO - Argument a15d135b-d67d-48e6-b1ec-ecec7179fde7 is now ModuleInstanceStatus.ADMITTED
2024-05-08 03:06:41,060 - rush - INFO - Argument 28e1b6f7-0ec8-4894-a592-7b9e451859c5 is now ModuleInstanceStatus.ADMITTED
2024-05-08 03:06:41,080 - rush - INFO - Argument 9070ebb6-b189-4012-b183-84fa777fa813 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:06:41,192 - rush - INFO - Argument 934dff6e-de42-4d92-86bf-7d6f637c3a46 is now ModuleInstanceStatus.ADMITTED
2024-05-08 03:06:41,235 - rush - INFO - Argument 29537b4a-f03c-4ae4-8215-a0ecf354b42c is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:06:41,268 - rush - INFO - Argument 25954215-051f-4b87-9f68-d4d7ce924b2d is now ModuleInstanceStatus.ADMITTED
2024-05-08 03:06:41,279 - rush - INFO - Argument cab1cb0a-6850-4ecd-8bec-cc56afd35c91 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03

[PosixPath('objects/8795952096450_rxdock.sdf'),
 PosixPath('objects/8795952092874_rxdock.sdf'),
 PosixPath('objects/8796043211371_rxdock.sdf'),
 PosixPath('objects/8795952096445_rxdock.sdf'),
 PosixPath('objects/8795952075116_rxdock.sdf'),
 PosixPath('objects/8795952075621_rxdock.sdf'),
 PosixPath('objects/8795952096435_rxdock.sdf'),
 PosixPath('objects/8795952096460_rxdock.sdf'),
 PosixPath('objects/8795952096465_rxdock.sdf'),
 PosixPath('objects/8795952075161_rxdock.sdf')]

In [ ]:
gnina_results = []

for pose in molecule_hashes:
    (docked_ligands, results) = await client.gnina_pdb(
        PROTEIN_TARGET_FILEPATH,
        Path.cwd() / 'objects' / f'{pose}_rxdock.sdf',
        Path.cwd() / f'{pose}.sdf',
        {
            "num_modes": 10,
            "exhaustiveness": 8,
            "minimise": True
        },
    )
    gnina_results.append(
        (pose, results, docked_ligands)
    )

await asyncio.gather(*(
    [output[1].get() for output in gnina_results]
    )
)

await asyncio.gather(*
        (
            [output[2].download(filename=f"{output[0]}_gnina.sdf", overwrite=True) for output in gnina_results]
        )
)

2024-05-08 03:11:31,689 - rush - INFO - Argument a409da26-4b25-4042-8bb6-58bb2d06eab2 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,738 - rush - INFO - Argument 5763049a-5288-45a2-bac3-883e06db323a is now ModuleInstanceStatus.RUNNING
2024-05-08 03:11:31,752 - rush - INFO - Argument ca9f1842-46ec-402a-82d4-33600967cf6f is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,753 - rush - INFO - Argument d381f225-cbfe-488e-814b-8961912fe964 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,760 - rush - INFO - Argument 3e7139cb-4e59-4831-81f4-09665cded126 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,769 - rush - INFO - Argument 11e255d6-97ed-49a7-a38c-a4ce174600ca is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,772 - rush - INFO - Argument 65a1e2d9-2371-46ac-ac77-fb35cf58abe2 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:11:31,783 - rush - INFO - Argument 8d58805b-9ffc-44af-8fb4-cb304e2a887b is now ModuleInstanceStatus.AWAITING_UPLOAD
2024

[PosixPath('objects/8795952096450_gnina.sdf'),
 PosixPath('objects/8795952092874_gnina.sdf'),
 PosixPath('objects/8796043211371_gnina.sdf'),
 PosixPath('objects/8795952096445_gnina.sdf'),
 PosixPath('objects/8795952075116_gnina.sdf'),
 PosixPath('objects/8795952075621_gnina.sdf'),
 PosixPath('objects/8795952096435_gnina.sdf'),
 PosixPath('objects/8795952096460_gnina.sdf'),
 PosixPath('objects/8795952096465_gnina.sdf'),
 PosixPath('objects/8795952075161_gnina.sdf')]

## 1.4) Sorting and presentation of results
In this section, we sort and display the top scored molecules from our virtual screen, including the QDXF conformers for each of our poses.

In [ ]:
POSE = 0
GNINA_SCORES = 1
GNINA_SDF_RESULTS = 2

In [ ]:
# Destructure the gnina scores from the Rush output
gnina_results = [(result[POSE], result[GNINA_SCORES].value, result[GNINA_SDF_RESULTS]) for result in gnina_results]

def find_best_pose(sublist, sort_key=SORT_BY):
    return max(item[sort_key.value] for item in sublist[1])

sorted_hits = sorted(gnina_results, key=find_best_pose, reverse=True)

In [ ]:
posed_conformers = []
for result in gnina_results:
    (conformers,) = await client.convert('SDF', result[GNINA_SDF_RESULTS])
    posed_conformers.append(
        (result[POSE], conformers)
    )
await asyncio.gather(*
        (
            [output[1].download(filename=f"{output[0]}_gnina_conformer.json", overwrite=True) for output in posed_conformers]
        )
)

2024-05-08 03:34:37,085 - rush - INFO - Argument 8e19bf18-390d-486d-92d9-88f25ea7373c is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,492 - rush - INFO - Argument 38b35356-8509-4738-ae83-f1a93f818766 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,536 - rush - INFO - Argument 28fd5c49-f308-4386-b492-de5bc515dab7 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,538 - rush - INFO - Argument 8359a4dd-e05e-4004-9c97-00547575f2da is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,539 - rush - INFO - Argument 5e3f0c9c-419e-4beb-81b6-81bb0b30e55b is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,565 - rush - INFO - Argument f263da53-d976-4254-95e6-b1384ead46c5 is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,570 - rush - INFO - Argument 46cd7219-04a3-4c43-b231-d111a353d0ef is now ModuleInstanceStatus.RESOLVING
2024-05-08 03:34:37,579 - rush - INFO - Argument e6c6c609-7b26-481f-a81c-40b167ca3881 is now ModuleInstanceStatus.RESOLVING
2024-05-

[PosixPath('objects/8795952096450_gnina_conformer.json'),
 PosixPath('objects/8795952092874_gnina_conformer.json'),
 PosixPath('objects/8796043211371_gnina_conformer.json'),
 PosixPath('objects/8795952096445_gnina_conformer.json'),
 PosixPath('objects/8795952075116_gnina_conformer.json'),
 PosixPath('objects/8795952075621_gnina_conformer.json'),
 PosixPath('objects/8795952096435_gnina_conformer.json'),
 PosixPath('objects/8795952096460_gnina_conformer.json'),
 PosixPath('objects/8795952096465_gnina_conformer.json'),
 PosixPath('objects/8795952075161_gnina_conformer.json')]

In [ ]:
final_output = []
for result in gnina_results:
    mhash = result[POSE]
    file_path = Path.cwd()/ 'objects' / f'{mhash}_gnina_conformer.json'
    conformers = json.loads(file_path.read_text())
    scores = result[GNINA_SCORES]
    final_output.append(
        (
            mhash,
            molecule_hashes[mhash],
            [{**i, **j} for i, j in zip(conformers, scores)]
         )
    )

## 1.4.1) Final table presentation
In this output, we display the outputs of our virtual screen. `rxdock` produces 10 poses per target ligand, but this protocol only displays only 2 poses by default.

In [ ]:
N_TOP_HITS = 10
N_POSES_PER_MOLECULE = 2

for molecule in final_output[0:N_TOP_HITS]:
    print("=" * 50)
    print(f"{MolToSmiles(molecule[1])}, hash: {molecule[0]}")

    for idx, pose in enumerate(molecule[2][0:N_POSES_PER_MOLECULE]):
        print(f"POSE {idx}")
        print(f"CNN score: {pose['cnn_score']}")
        print(f"Affinity: {pose['affinity']}")
        print(f"CNN affinity: {pose['cnn_affinity']}")
        # print(f'{pose['topology']['symbols'][0:5]}') if you want to inspect the output QDXF conformer
        print("-" * 50)

[H]OC(=O)C([H])([H])N([H])C(=O)[C@]([H])(S[H])C([H])([H])[H], hash: 8795952096450
POSE 0
CNN score: 0.5908319
Affinity: -4.52479
CNN affinity: 3.984918
['C', 'O', 'N', 'H', 'C']
--------------------------------------------------
POSE 1
CNN score: 0.5909194
Affinity: -4.52488
CNN affinity: 3.9846313
['C', 'O', 'N', 'H', 'C']
--------------------------------------------------
[H]OC(=O)C([H])([H])N([H])C(=O)[C@]([H])(S[H])C([H])([H])[H], hash: 8795952092874
POSE 0
CNN score: 0.3093567
Affinity: -4.09026
CNN affinity: 3.4164128
['C', 'O', 'N', 'H', 'C']
--------------------------------------------------
POSE 1
CNN score: 0.35947412
Affinity: -4.16741
CNN affinity: 3.3598225
['C', 'O', 'N', 'H', 'C']
--------------------------------------------------
[H]OC(=O)C([H])([H])N([H])C(=O)[C@]([H])(S[H])C([H])([H])[H], hash: 8796043211371
POSE 0
CNN score: 0.5947218
Affinity: -4.21522
CNN affinity: 3.6771526
['C', 'O', 'N', 'H', 'C']
--------------------------------------------------
POSE 1
CNN sco